# Langchain

In [45]:
import os
from dotenv import load_dotenv
load_dotenv()

True

## LCEL (LangChain Expression Language)
- LangChain 重写了 `|`（`__or__`），`Chain 之所在`\n",
- `RunnablePassthrough`: RunnablePassthrough 允许你将输入数据直接传递而不做任何更改（identity），通常与 RunnableParallel 一起使用，将数据传递到新的键中。\n",
- LangChain 的应用 RunnablePassthrough 作为一个占位符，可以在需要时填充数据，比如在公司名称尚未确定时先留空，后续再填入。\n",
- 所谓的最佳实践\n",
    - python：遍历，也可以用 list comprehension\n",
    - 对于 matlab：也可以遍历，也可以整理成 matrix，直接矩阵矢量乘法；

In [2]:
2 | 3 > 2

True

In [3]:
2 | 3

3

In [4]:
from langchain_core.runnables import (
    RunnablePassthrough, 
    RunnableLambda, 
    RunnableParallel
)
os.environ["LANGCHAIN_PROJECT"] = 'lcel_test'

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
output_parser = StrOutputParser()
llm = ChatOpenAI(model="gpt-3.5-turbo")

# lcel
chain = (
    {"topic": RunnablePassthrough()} 
    | prompt
    | llm
    | output_parser
)

chain.invoke("ice cream")


'Why did the ice cream cone go to therapy? Because it had too many toppings!'

### Runables
- `RunnablePassthrough()`: identity
- `RunnableLambda()`: function

In [7]:
chain = RunnablePassthrough() | RunnablePassthrough () | RunnablePassthrough ()
chain.invoke("hello")

'hello'

In [8]:
chain = RunnablePassthrough() | RunnableLambda(lambda x: x.upper())
chain.invoke("hello")

'HELLO'

In [9]:
chain = RunnablePassthrough() | RunnableLambda(lambda x: x.upper()) | RunnablePassthrough()
chain.invoke("hello")

'HELLO'

### json test

In [10]:
os.environ["LANGCHAIN_PROJECT"] = 'json_test2'

In [11]:
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.prompts.chat import SystemMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser

In [12]:
llm = ChatOpenAI(model="gpt-4o", 
                 model_kwargs={'response_format': {"type": "json_object"}})

In [13]:
json_parser = JsonOutputParser()

In [14]:
# 创建提示模板
prompt = ChatPromptTemplate.from_messages([
    ("system", '''I want you to extract the person name, age and a description from the following text.
    Here is the JSON object, output:
    {{
        "name": string,
        "age": int,
        "description": string
    }}'''),
    ("human", "{input}")
])

# 创建 LCEL 链
chain = (
    {"input": RunnablePassthrough()} 
    | prompt 
    | llm 
    | json_parser
)

In [15]:
prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='I want you to extract the person name, age and a description from the following text.\n    Here is the JSON object, output:\n    {{\n        "name": string,\n        "age": int,\n        "description": string\n    }}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [16]:
print(prompt[0])
print(prompt[1])

prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='I want you to extract the person name, age and a description from the following text.\n    Here is the JSON object, output:\n    {{\n        "name": string,\n        "age": int,\n        "description": string\n    }}') additional_kwargs={}
prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}') additional_kwargs={}


In [17]:
ChatPromptTemplate.from_messages(
    [SystemMessagePromptTemplate.from_template('''I want you to extract the person name, age and a description from the following text.
    Here is the JSON object, output:
    {{
        "name": string,
        "age": int,
        "description": string
    }}'''), 
     HumanMessagePromptTemplate.from_template("{input}")
    ]
)

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='I want you to extract the person name, age and a description from the following text.\n    Here is the JSON object, output:\n    {{\n        "name": string,\n        "age": int,\n        "description": string\n    }}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [18]:
result = chain.invoke("John is 20 years old. He is a student at the University of California, Berkeley. He is a very smart student.")

In [20]:
result

{'name': 'John',
 'age': 20,
 'description': 'He is a student at the University of California, Berkeley. He is a very smart student.'}

In [21]:
chain.invoke({'input': "John is 20 years old. He is a student at the University of California, Berkeley. He is a very smart student."})

{'name': 'John',
 'age': 20,
 'description': 'He is a student at the University of California, Berkeley. He is a very smart student.'}

### RAG

In [6]:
os.environ["LANGCHAIN_PROJECT"] = 'rag_test'

In [22]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [23]:
OpenAIEmbeddings()

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x0000016320049F90>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000016320049E70>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [24]:
vectorstore = FAISS.from_texts(["Cats love thuna"], embedding = OpenAIEmbeddings())

In [25]:
retriever = vectorstore.as_retriever()

In [26]:
retriever.invoke("What do cats like to eat?")

[Document(id='b6f9cf5a-8e5b-48db-9870-ae34b6d7f59f', metadata={}, page_content='Cats love thuna')]

In [28]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template=template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | ChatOpenAI()
    | StrOutputParser()
)

In [29]:
rag_chain.invoke("What do cats like to eat?")

'Tuna'

### Tool uses
- precise math calculation
- custom tools (自定义 functions)

In [30]:
os.environ["LANGCHAIN_PROJECT"] = 'tools_test2'

In [31]:
import numpy as np
from langchain_core.tools import Tool
from langchain_core.tools import tool

In [38]:
@tool
def add(num1: float, num2: float) -> float:
    "Add two numbers."
    return num1 + num2
    
@tool
def subtract(num1: float, num2: float) -> float:
    """
    Subtract two numbers.
    """
    return num1 - num2
    
@tool
def multiply(num1: float, num2: float) -> float:
    """Multiply two float ."""
    return num1 * num2

@tool
def divide(numerator: float, denominator: float) -> float:
    """
    Divides the numerator by the denominator.
    """

    result = numerator / denominator
    return result

@tool
def power(base: float, exponent: float) -> float:
    "Take the base to the exponent power, base^exponent."
    return base**exponent


@tool
def exp(x):
    """
    Calculate the natural exponential $e^x$
    """
    return np.exp(x)

In [34]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

In [41]:
tools = [add, subtract, multiply, divide, power, exp]

llm = ChatOpenAI(model="gpt-4o", temperature=0, streaming=True)

system_template = """
You are a helpful math assistant that uses calculation functions to solve complex math problems step by step.
"""

human_template = "{input}"

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_template),
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        HumanMessagePromptTemplate.from_template(input_variables=["input"], template=human_template),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent = create_openai_tools_agent(llm, tools, prompt)

In [42]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [43]:
agent_executor.invoke({"input": "What is the result of directive of sigmoid(2.5)?"})



> Entering new AgentExecutor chain...

Invoking: `exp` with `{'x': -2.5}`
responded: The derivative of the sigmoid function, often denoted as \(\sigma(x)\), is given by the formula:

\[
\sigma'(x) = \sigma(x) \cdot (1 - \sigma(x))
\]

where \(\sigma(x) = \frac{1}{1 + e^{-x}}\).

To find the derivative of the sigmoid function at \(x = 2.5\), we need to:

1. Calculate \(\sigma(2.5)\).
2. Use the derivative formula \(\sigma'(x) = \sigma(x) \cdot (1 - \sigma(x))\).

Let's start by calculating \(\sigma(2.5)\):

\[
\sigma(2.5) = \frac{1}{1 + e^{-2.5}}
\]

I'll calculate \(e^{-2.5}\) first.

0.0820849986238988
Invoking: `add` with `{'num1': 1, 'num2': 0.0820849986238988}`
responded: The value of \(e^{-2.5}\) is approximately \(0.0821\).

Now, we can calculate \(\sigma(2.5)\):

\[
\sigma(2.5) = \frac{1}{1 + 0.0821}
\]

Let's compute this value.

1.0820849986238987
Invoking: `divide` with `{'numerator': 1, 'denominator': 1.0820849986238987}`


0.9241418199787566
Invoking: `subtract` with `{'n

{'input': 'What is the result of directive of sigmoid(2.5)?',
 'output': "The derivative of the sigmoid function at \\(x = 2.5\\), \\(\\sigma'(2.5)\\), is approximately \\(0.0701\\)."}

In [46]:
import torch
import torch.nn.functional as F

In [47]:
F.sigmoid(torch.tensor([2.5])) * (1-F.sigmoid(torch.tensor([2.5])))

tensor([0.0701])